In [1]:
import json

import numpy as np
import pandas as pd
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

In [2]:
with open('db_curated_server_info.txt', 'r') as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()  
    if line and not line.startswith('#'):  
        key, value = line.split('=', 1)  
        config[key] = value.strip()  

server_name = config.get('server_name')
srever_port = int(config.get('srever_port'))
ssh_password = config.get('ssh_password')
ssh_username = config.get('ssh_username')
db_adress = config.get('db_adress')
db_port = int(config.get('db_port'))

In [3]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

43881


In [4]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [5]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [6]:
query = "SELECT * FROM histone"
cursor.execute(query)
cursor.fetchall()
", h.".join([i[0] for i in cursor.description])

'id, h.level, h.taxonomic_span, h.taxonomic_span_id, h.description, h.parent'

In [7]:
query = "SELECT * FROM histone_description"
cursor.execute(query)
cursor.fetchall()
", d.".join([i[0] for i in cursor.description])

'id, d.summary, d.taxonomy, d.genes, d.evolution, d.expression, d.knock_out, d.function, d.sequence, d.localization, d.deposition, d.structure, d.interactions, d.disease, d.caveats'

In [8]:
query = "SELECT * FROM alternative_name"
cursor.execute(query)
cursor.fetchall()
", an.".join([i[0] for i in cursor.description])

'id, an.name, an.taxonomy, an.gene, an.splice, an.histone'

In [9]:
query = "SELECT * FROM publication"
cursor.execute(query)
cursor.fetchall()
", p.".join([i[0] for i in cursor.description])

'id, p.title, p.doi, p.author, p.year'

In [10]:
query = "SELECT * FROM sequence"
cursor.execute(query)
cursor.fetchall()
", p.".join([i[0] for i in cursor.description])

'accession, p.variant, p.gi, p.ncbi_gene_id, p.hgnc_gene_name, p.taxonomy_id, p.organism, p.phylum, p.class, p.taxonomy_group, p.info, p.sequence, p.variant_under_consideration'

In [11]:
query = "SELECT * FROM histone WHERE id='H3.3'"
cursor.execute(query)
# for i in cursor:
#     print(i)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df

,id,level,taxonomic_span,taxonomic_span_id,description,parent
0,H3.3,variant_group,Eukaryotes,2759,49,H3


In [12]:
query = "SELECT id FROM histone WHERE parent IS NULL"
cursor.execute(query)
types = cursor.fetchall()
types

[('Archaeal',), ('H1',), ('H2A',), ('H2B',), ('H3',), ('H4',), ('Viral',)]

In [7]:
def get_tree_dict(cursor, paretnt_id):
    cursor.execute(f"SELECT id FROM histone WHERE parent = '{paretnt_id}'")
    res = cursor.fetchall()
    if len(res) < 1:
        return "null"
    return {v: get_tree_dict(cursor, v) for v, *_ in res}

In [8]:
def dict_clean(items, default="null"):
    result = {}
    for key, value in items.items():
        if value is None:
            if default is None:
                continue
            value = default
        result[key] = value
    return result

In [9]:
def replace_key_in_dict(data, old_key, new_key):
    if isinstance(data, dict):
        new_data = {}
        for key, value in data.items():
            # Если ключ совпадает с `old_key`, заменяем его на `new_key`
            if key == old_key:
                new_data[new_key] = replace_key_in_dict(value, old_key, new_key)
            else:
                new_data[key] = replace_key_in_dict(value, old_key, new_key)
        return new_data
    elif isinstance(data, list):
        # Рекурсивно обрабатываем списки
        return [replace_key_in_dict(item, old_key, new_key) for item in data]
    return data

# Save to JSON

In [16]:
%%time
query = "SELECT id FROM histone WHERE parent IS NULL"
cursor.execute(query)
types = cursor.fetchall()
variants_tree = {}
for t, *_ in types:
    variants_tree[t] = get_tree_dict(cursor, t)

variants_info = {}
query = (
    "SELECT hdap.*, p.title, p.doi, p.author, p.year FROM ( "
    "SELECT hda.*, publication_id FROM ( "
    "SELECT hd.*, an.name, an.taxonomy as an_taxonomy, an.gene, an.splice FROM ( "
    "SELECT h.id, "
    "h.level,"
    "h.taxonomic_span, "
    "h.taxonomic_span_id, "
    "h.parent, "
    "d.summary, "
    "d.taxonomy, "
    "d.genes, "
    "d.evolution, "
    "d.expression, "
    "d.knock_out, "
    "d.function, "
    "d.sequence, "
    "d.localization, "
    "d.deposition,"
    "d.structure,"
    "d.interactions, "
    "d.disease, "
    "d.caveats FROM histone h LEFT JOIN histone_description d ON h.description = d.id "
    ") hd LEFT JOIN alternative_name an ON hd.id = an.histone "
    ") hda LEFT JOIN histone_has_publication hp ON hda.id = hp.histone_id "
    ") hdap LEFT JOIN publication p ON hdap.publication_id = p.id;"
)
cursor.execute(query)
variants_info_df = pd.DataFrame(
    cursor.fetchall(), columns=[i[0] for i in cursor.description]
)
for var_name in variants_info_df.id.unique():
    # print(var_name)
    level_value = variants_info_df[variants_info_df["id"] == var_name].iloc[0]["level"]
    taxonomic_span_value = variants_info_df[variants_info_df["id"] == var_name].iloc[0][
        "taxonomic_span"
    ]
    taxonomic_span_id_value = variants_info_df[variants_info_df["id"] == var_name].iloc[
        0
    ]["taxonomic_span_id"]
    variants_info[var_name] = {
        "level": level_value if level_value else "null",
        "description": {},
        "taxonomic_span": taxonomic_span_value if taxonomic_span_value else "null",
        "taxonomic_span_id": taxonomic_span_id_value
        if taxonomic_span_id_value
        else "null",
        "alternate_names": [],
        "publications": [],
    }
    var_publications = (
        variants_info_df.groupby(["id"])["publication_id"]
        .apply(lambda x: [] if x.dropna().empty else list(x.unique()))
        .reset_index()
    )
    variants_info[var_name]["publications"] = var_publications[
        var_publications["id"] == var_name
    ].iloc[0]["publication_id"]
    var_names = (
        variants_info_df.groupby(["id"])[["name", "an_taxonomy", "gene", "splice"]]
        .apply(
            lambda x: []
            if x.drop_duplicates().dropna(how="all").empty
            else list(
                map(
                    lambda y: dict_clean(y, default=None),
                    x.drop_duplicates()
                    .rename(columns={"an_taxonomy": "taxonomy"})
                    .to_dict("records"),
                )
            )
        )
        .reset_index()
    )
    variants_info[var_name]["alternate_names"] = var_names[
        var_names["id"] == var_name
    ].iloc[0][0]
    default_description = {
        "summary": "null",
        "taxonomy": "null",
        "genes": "null",
        "evolution": "null",
        "expression": "null",
        "knock-out": "null",
        "function": "null",
        "sequence": "null",
        "localization": "null",
        "deposition": "null",
        "structure": "null",
        "interactions": "null",
        "disease": "null",
        "caveats": "null",
    }
    var_desc = (
        variants_info_df.groupby(["id"])[
            [
                "summary",
                "taxonomy",
                "genes",
                "evolution",
                "expression",
                "knock_out",
                "function",
                "sequence",
                "localization",
                "deposition",
                "structure",
                "interactions",
                "disease",
                "caveats",
            ]
        ]
        .apply(
            lambda x: default_description
            if x.drop_duplicates().dropna(how="all").empty
            else dict_clean(x.drop_duplicates().to_dict("records")[0])
        )
        .reset_index()
    )
    variants_info[var_name]["description"] = var_desc[var_desc["id"] == var_name].iloc[
        0
    ][0]
    # Заменяем ключи во всех словарях
    for key, value in variants_info.items():
        variants_info[key] = replace_key_in_dict(value, "knock_out", "knock-out")

with open("sample.json", "w") as outfile:
    json.dump({"tree": variants_tree, "info": variants_info}, outfile, indent=4)

CPU times: user 7min 36s, sys: 8.55 s, total: 7min 45s
Wall time: 7min 24s


## Testing queries

In [394]:
query = (
    "SELECT hdap.*, p.title, p.doi, p.author, p.year FROM ( "
    "SELECT hda.*, publication_id FROM ( "
    "SELECT hd.*, an.name, an.taxonomy as an_taxonomy, an.gene, an.splice FROM ( "
    "SELECT h.id, "
    "h.level,"
    "h.taxonomic_span, "
    "h.taxonomic_span_id, "
    "h.parent, "
    "d.summary, "
    "d.taxonomy, "
    "d.genes, "
    "d.evolution, "
    "d.expression, "
    "d.knock_out, "
    "d.function, "
    "d.sequence, "
    "d.localization, "
    "d.deposition,"
    "d.structure,"
    "d.interactions, "
    "d.disease, "
    "d.caveats FROM histone h LEFT JOIN histone_description d ON h.description = d.id "
    ") hd LEFT JOIN alternative_name an ON hd.id = an.histone "
    ") hda LEFT JOIN histone_has_publication hp ON hda.id = hp.histone_id "
    ") hdap LEFT JOIN publication p ON hdap.publication_id = p.id;"
)
cursor.execute(query)
test_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
# test_df.columns
# test_df.groupby(["id"])["publication_id"].get_group("Archaeal").dropna().empty
# test_df2 = (
#     test_df.groupby(["id"])["publication_id"]
#     .apply(lambda x: [] if x.dropna().empty else list(x.unique()))
#     .reset_index()
# )
# test_df2[test_df2["id"] == "macroH2A"].iloc[0]["publication_id"]

# test_df.groupby(["id"])["summary"].apply(lambda x: pd.unique(x)).reset_index()
# default_description = {
#     "summary": "null",
#     "taxonomy": "null",
#     "genes": "null",
#     "evolution": "null",
#     "expression": "null",
#     "knock-out:": "null",
#     "function": "null",
#     "sequence": "null",
#     "localization": "null",
#     "deposition": "null",
#     "structure": "null",
#     "interactions": "null",
#     "disease": "null",
#     "caveats": "null"
# }
# test_df.groupby(["id"])[
#     [
#         "summary",
#         "taxonomy",
#         "genes",
#         "evolution",
#         "expression",
#         "knock_out",
#         "function",
#         "sequence",
#         "localization",
#         "deposition",
#         "structure",
#         "interactions",
#         "disease",
#         "caveats",
#     ]
# ].apply(
#     lambda x: default_description
#     if x.drop_duplicates().dropna(how="all").empty
#     else dict_clean(x.drop_duplicates().to_dict("records")[0])
# ).reset_index()

test_df.groupby(["id", "name"]).get_group(("macroH2A", "macroH2A1"))
test_df.groupby(["id"])[["name", "an_taxonomy", "gene", "splice"]].get_group(
    "macroH2A"
).drop_duplicates().dropna(how="all").empty
test_df2 = (
    test_df.groupby(["id"])[["name", "an_taxonomy", "gene", "splice"]]
    .apply(
        lambda x: []
        if x.drop_duplicates().dropna(how="all").empty
        else list(
            map(
                lambda y: dict_clean(y, default=None),
                x.drop_duplicates()
                .rename(columns={"an_taxonomy": "taxonomy"})
                .to_dict("records"),
            )
        )
    )
    .reset_index()
)
test_df2
# test_df2[test_df2["id"]=="Archaeal"][0].iloc[0]
# del test_df2, test_df

,id,0
0,Archaeal,[]
1,CS_H2B_(Echinoidea),[{'name': 'cleavage H2B'}]
2,H1,[]
3,H1.0,"[{'name': 'H1°'}, {'name': 'H5', 'taxonomy': '..."
4,H1.1,[]
...,...,...
178,macroH2A.2_(Homo_sapiens),[]
179,macroH2A.2_(Mammalia),[]
180,scH1,[]
181,short_H2A,[{'name': 'sH2A'}]


In [140]:
query = "SELECT * FROM sequence_has_publication"
cursor.execute(query)
cursor.fetchall()
[i[0] for i in cursor.description]

['sequence_accession', 'publication_id']

## Сравним полученный JSON с classification.json

**Вывод:**
То, что лежит в переменной `differences` под ключами `'dictionary_item_removed'` и `'values_changed'` необходимо добавить в БД и заново запустить скрипт из данного ноутбука.

In [295]:
!pip install deepdiff

In [16]:
def transform_dict(info):
    # Шаблон для полей description
    default_description = {
        "summary": "null",
        "taxonomy": "null",
        "genes": "null",
        "evolution": "null",
        "expression": "null",
        "knock-out": "null",
        "function": "null",
        "sequence": "null",
        "localization": "null",
        "deposition": "null",
        "structure": "null",
        "interactions": "null",
        "disease": "null",
        "caveats": "null",
    }

    # Проверка, если description является строкой
    if isinstance(info.get("description"), str):
        # Сохраняем значение summary из строки description
        summary_text = info["description"]
        # Заменяем строку на словарь с заполненными значениями по умолчанию
        info["description"] = default_description.copy()
        # Присваиваем значение summary из исходной строки
        info["description"]["summary"] = summary_text

    return info

In [17]:
#  удалим -- и заменим кавычки так, как это представлено в БД
def clean_summary(data):
    if isinstance(data, dict):
        for key, value in data.items():
            # Если ключ 'summary', выполняем замены
            if key == "summary" and isinstance(value, str):
                data[key] = value.replace("--", "")  # .replace("'", "''")
            else:
                # Рекурсивно обрабатываем вложенные словари
                clean_summary(value)
    elif isinstance(data, list):
        # Рекурсивно обрабатываем списки
        for item in data:
            clean_summary(item)

In [18]:
# добавляем пропущенные поля в 'description'
def add_missing_fields(info, required_fields):
    for key, value in info.items():
        if isinstance(value, dict):
            # Проверяем наличие поля 'description'
            if "description" in value:
                # Проверяем наличие необходимых полей в 'description'
                for field in required_fields:
                    if field not in value["description"]:
                        value["description"][
                            field
                        ] = "null"  # Или любое другое значение по умолчанию
            # Рекурсивно обрабатываем вложенные словари
            add_missing_fields(value, required_fields)

In [19]:
with open(
    "/home/l_singh/_scratch/hdb/project_dir/histonedb/CURATED_SET/classification.json",
    "r",
    encoding="utf-8",
) as file:
    data1 = json.load(file)
# Преобразуем каждый элемент в словаре data1['info']
for key, value in data1["info"].items():
    data1["info"][key] = transform_dict(value)
for key, value in data1.items():
    data1[key] = replace_key_in_dict(value, "knock-out:", "knock-out")
clean_summary(data1["info"])
# Добавляем недостающие поля
add_missing_fields(data1, list(default_description.keys())[1:])
# with open("/home/l_singh/_scratch/hdb/project_dir/histonedb/CURATED_SET/classification.json", 'w', encoding='utf-8') as file:
#     json.dump(data1, file)

with open("sample.json", "r", encoding="utf-8") as file:
    data2 = json.load(file)

In [20]:
data1["info"]["cH4"]

{'level': 'variant_group',
 'description': {'summary': 'Canonical histones are replication-dependent while histone variants are replication-independent, constitutively expressed during cell cycle. Genes encoding canonical histones are typically located within multigene clusters and use specific type of regulation at the RNA level with a stem loop structure instead of polyA tail',
  'taxonomy': 'null',
  'genes': 'null',
  'evolution': 'null',
  'expression': 'null',
  'knock-out': 'null',
  'function': 'null',
  'sequence': 'null',
  'localization': 'null',
  'deposition': 'null',
  'structure': 'null',
  'interactions': 'null',
  'disease': 'null',
  'caveats': 'null'},
 'taxonomic_span': 'Eukaryotes',
 'taxonomic_span_id': '2759',
 'alternate_names': [{'name': 'ca H4'}],
 'publications': ['22650316']}

In [21]:
data1["info"]["H3.B_(Giardia?)"]

{'level': 'variant_group',
 'description': {'summary': 'null',
  'taxonomy': 'null',
  'genes': 'null',
  'evolution': 'null',
  'expression': 'null',
  'knock-out': 'null',
  'function': 'null',
  'sequence': 'null',
  'localization': 'null',
  'deposition': 'null',
  'structure': 'null',
  'interactions': 'null',
  'disease': 'null',
  'caveats': 'null'},
 'taxonomic_span': 'Giardia',
 'taxonomic_span_id': '5740',
 'alternate_names': [],
 'publications': []}

In [22]:
data2["info"]["H3.B_(Giardia?)"]

{'level': 'variant_group',
 'description': {'summary': 'null',
  'taxonomy': 'null',
  'genes': 'null',
  'evolution': 'null',
  'expression': 'null',
  'knock-out': 'null',
  'function': 'null',
  'sequence': 'null',
  'localization': 'null',
  'deposition': 'null',
  'structure': 'null',
  'interactions': 'null',
  'disease': 'null',
  'caveats': 'null'},
 'taxonomic_span': 'Giardia',
 'taxonomic_span_id': '5740',
 'alternate_names': [],
 'publications': []}

In [23]:
data2["info"]["cH2A"]["alternate_names"]

[{'name': 'canonical H2A'}]

In [24]:
data2["info"]["cH2A_(Mus_musculus)"]["alternate_names"]

[]

In [25]:
data2["info"]["H1.0"]["alternate_names"]

[{'name': 'H1°'},
 {'name': 'H5', 'taxonomy': 'aves'},
 {'name': 'H1δ'},
 {'name': 'RI H1'}]

In [26]:
data1["info"]["cH2A.10_(Homo_sapiens)"]["description"]

{'summary': 'cH2A.10_(Homo_sapiens)  is an isoform (variant) of clustered (canonical) H2A histones in human endoded by H2AC21 gene.',
 'taxonomy': 'null',
 'genes': 'null',
 'evolution': 'null',
 'expression': 'null',
 'knock-out': 'null',
 'function': 'null',
 'sequence': 'null',
 'localization': 'null',
 'deposition': 'null',
 'structure': 'null',
 'interactions': 'null',
 'disease': 'null',
 'caveats': 'null'}

In [27]:
# Функция для приведения всех значений к строкам (рекурсивно)
def convert_values_to_str(data):
    if isinstance(data, dict):
        return {key: convert_values_to_str(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_values_to_str(item) for item in data]
    elif isinstance(
        data, (int, float, bool)
    ):  # Преобразуем числа и логические значения в строки
        return str(data)
    return data  # Возвращаем неизмененные строковые значения

In [28]:
# Приводим все значения к строкам для корректного сравнения
data1 = convert_values_to_str(data1)
data2 = convert_values_to_str(data2)

In [31]:
from deepdiff import DeepDiff

# Используем DeepDiff для сравнения
differences = DeepDiff(data1, data2, ignore_order=True)

# Если есть различия, выводим их
if differences:
    print("Найдены различия:")
    print("dictionary_item_added:")
    print(differences["dictionary_item_added"])
    # print("dictionary_item_removed:")
    # print(differences['dictionary_item_removed'])
    # print("values_changed:")
    # for k in list(differences['values_changed'].keys()):
    #     if 'level' in k or 'taxonomic_span' in k: continue
    #     print(f"{k}: {differences['values_changed'][k]}")
else:
    print("Файлы идентичны.")

# with open("sample_diff.json", "w") as outfile:
#     json.dump(differences, outfile)

Найдены различия:
dictionary_item_added:
SetOrdered(["root['tree']['H2B']['cH2B']['cH2B_(Animals)']['cH2B_(Vertebrata)']['cH2B_(Mammalia)']['cH2B_(Homo_sapiens)']['cH2B.15_(Homo_sapiens)']", "root['info']['Archaeal']", "root['info']['cenH3_(Eukarya)']", "root['info']['cH1']", "root['info']['H2A.J_(Homo_sapiens)']", "root['info']['H2A.X_(Animals)']", "root['info']['H2A.X_(Fungi)']", "root['info']['H2A.X_(Homo_sapiens)']", "root['info']['H2A.X_(Mammalia)']", "root['info']['H2A.X_(Mus_musculus)']", "root['info']['H2A.X_(Plants)']", "root['info']['H2A.X_(Protists)']", "root['info']['H2A.X_(Vertebrata)']", "root['info']['H3.1-like_(Plants)']", "root['info']['H3.3-like?']", "root['info']['H3.3-like_(Animals)']", "root['info']['H3.3-like_(Plants)']", "root['info']['H3.5_(Primates_or_Hominids?)']", "root['info']['H3.6_(Mammals?)?']", "root['info']['H3.7_(Mammals?)?']", "root['info']['H3.8_(Mammals?)?']", "root['info']['H3.X_(Homo_sapiens)']", "root['info']['H3.X_(Primates?)']", "root['info']['

In [43]:
query = "SELECT * FROM alternative_name"
cursor.execute(query)
alternative_name_df = pd.DataFrame(
    cursor.fetchall(), columns=[i[0] for i in cursor.description]
)
alternative_name_df[alternative_name_df["histone"] == "macroH2A"]

,id,name,taxonomy,gene,splice,histone
9,10,mH2A,None,None,None,macroH2A
10,11,macroH2A1,None,None,None,macroH2A
11,12,macroH2A2,None,None,None,macroH2A
12,13,macroH2A1.1,None,None,None,macroH2A
13,14,macroH2A1.2,None,None,None,macroH2A
14,15,macroH2A2.1,None,None,None,macroH2A
15,16,macroH2A2.2,None,None,None,macroH2A


In [408]:
data2["info"]["cH2B.15_(Homo_sapiens)"]

KeyError: 'cH2B.15_(Homo_sapiens)'

В некоторых описаниях classification.json нет полея taxonomy, taxonomic_span_id

In [436]:
differences.keys()

dict_keys(['dictionary_item_added', 'dictionary_item_removed', 'values_changed'])

In [508]:
list(differences["values_changed"].keys())[28:]

["root['info']['cH3.1_(Mammalia)']['level']",
 "root['info']['cH3.2_(Mammalia)']['level']",
 "root['info']['cH3_(Animals)']['level']",
 "root['info']['cH3_(Chlorophyta)']['level']",
 "root['info']['cH3_(Chlorophyta)']['description']['summary']",
 "root['info']['cH3_(Embryophyta)']['level']",
 "root['info']['cH3_(Fungi)']['level']",
 "root['info']['cH3_(Mammalia)']['level']",
 "root['info']['cH3_(Plants)']['level']",
 "root['info']['cH3_(Plants)']['description']['summary']",
 "root['info']['cH3_(Protists)']['level']",
 "root['info']['cH3_(Vertebrata)']['level']",
 "root['info']['H2A.J']['description']['summary']",
 "root['info']['H2A.L']['description']['summary']",
 "root['info']['H2A.L.1_(Homo_sapiens)']['description']['summary']",
 "root['info']['H2A.L.3_(Homo_sapiens)']['description']['summary']",
 "root['info']['H2A.L_(Homo_sapiens)']['description']['summary']",
 "root['info']['H2A.P']['description']['summary']",
 "root['info']['H2A.P_(Homo_sapiens)']['description']['summary']",
 "r

# Save to CSV

In [10]:
def find_parent_by_level(df, variant_name, target_level):
    if variant_name == "":
        return ""

    # Ищем строку с данным названием варианта
    current_row = df[df["id"] == variant_name]
    if current_row.empty:
        return "HISTONE_NOT_EXIST"

    if current_row["level"].values[0] == target_level:
        return current_row["id"].values[0]

    # Пока не найден родитель с нужным уровнем
    while not current_row.empty:
        # Получаем значение поля parent
        parent_id = current_row["parent"].values[0]

        if pd.isna(parent_id):
            print("here")
            return None  # Нет родителя

        # Ищем родителя
        parent_row = df[df["id"] == parent_id]

        # Проверяем уровень родителя
        if parent_row["level"].values[0] == target_level:
            return parent_row["id"].values[0]

        # Продолжаем искать родителя на следующем уровне
        current_row = parent_row

    return None  # Если не найдено

In [11]:
query = "SELECT * FROM sequence s LEFT JOIN sequence_has_publication sp ON s.accession = sp.sequence_accession"
cursor.execute(query)
sequence_df = pd.DataFrame(
    cursor.fetchall(), columns=[i[0] for i in cursor.description]
)

sequence_df = (
    sequence_df.groupby(["accession"])
    .agg(lambda x: "" if x.dropna().empty else " ".join(list(map(str, x.unique()))))
    .reset_index()
)

sequence_df = sequence_df.drop(columns=["sequence_accession"])
sequence_df["taxonomy_id"] = (
    pd.to_numeric(sequence_df["taxonomy_id"], errors="coerce")
    .fillna(np.nan)
    .astype("Int64")
)

query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
# sequence_df["variant"] = [
#     row["variant"] if row["variant"] != "" else row["variant_under_consideration"]
#     for i, row in sequence_df.iterrows()
# ]
sequence_df["variant_group"] = sequence_df["variant"].apply(
    lambda x: find_parent_by_level(histone_df, x, "variant_group")
)
sequence_df["type"] = sequence_df["variant"].apply(
    lambda x: find_parent_by_level(histone_df, x, "type")
)

sequence_df["doublet"] = [""] * sequence_df.shape[0]

sequence_df = (
    sequence_df[
        [
            "accession",
            "type",
            "variant_group",
            "variant",
            # "variant_under_consideration",
            "doublet",
            "gi",
            "ncbi_gene_id",
            "hgnc_gene_name",
            "taxonomy_id",
            "organism",
            "phylum",
            "class",
            "taxonomy_group",
            "info",
            "publication_id",
            "sequence",
        ]
    ]
    .rename(columns={"publication_id": "references"})
    .replace("None", "")
)
sequence_df = sequence_df.sort_values(["variant_group", "variant", "phylum", "class", "organism"])
sequence_df.to_csv('histones_sample.csv', index=False)

sequence_df.head()

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
1,AAB48832.1,H2B,CS_H2B_(Echinoidea),CS_H2B_(Echinoidea),,,,,7660,Psammechinus miliaris,Echinodermata,Echinoidea,,,9032246,MPAKGAATKGEKKQAVKSKAMASSRTGDKKRRRRRLESYNIYIYKV...
37,EED88841.1,H1,H1.0,H1.0,,220970504,,,296543,Thalassiosira pseudonana CCMP1335,Bacillariophyta,Coscinodiscophyceae,,,,MSYKAGIAKAITELKDRTGSSSIAIKKHMQANLPADKKWMNATFLK...
525,XP_010887142.1,H1,H1.0,H1.0,,742177175,,,8010,Esox lucius,Chordata,Actinopteri,,,,MAETVAAPAPKAKKAKAPKKPASHPKYSDMIKAAVQADKSRGGASR...
10,ACH70944.1,H1,H1.0,H1.0,,197632441,,,8030,Salmo salar,Chordata,Actinopteri,,,,MAETAAAPAPKAKKAKAPKKPASHPKYSDMIKAAVHADKSRGGASR...
12,ACM08534.1,H1,H1.0,H1.0,,221219746,,,8030,Salmo salar,Chordata,Actinopteri,,,,MIKAAVHADKSRGGASRQSVQKYIKSHYKVGDNADSQIKLSLKRMV...


In [12]:
sequence_df[sequence_df["variant_group"] == "HISTONE_NOT_EXIST"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence


In [13]:
sequence_df.isna().sum(axis=0)

accession          0
type               0
variant_group      0
variant            0
doublet            0
gi                 0
ncbi_gene_id       0
hgnc_gene_name     0
taxonomy_id       20
organism           0
phylum             0
class              0
taxonomy_group     0
info               0
references         0
sequence           0
dtype: int64

In [14]:
sequence_df[sequence_df["phylum"] == "None"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence


## Testing queries

In [37]:
query = "SELECT * FROM sequence s LEFT JOIN sequence_has_publication sp ON s.accession = sp.sequence_accession"
cursor.execute(query)
sequence_df = pd.DataFrame(
    cursor.fetchall(), columns=[i[0] for i in cursor.description]
)
sequence_df.groupby(["accession"]).get_group("AAC28846.1")

sequence_df.groupby(["accession"]).agg(
    lambda x: "" if x.dropna().empty else " ".join(list(map(str, x.unique())))
).reset_index().to_csv("test.csv")

In [77]:
def find_parent_by_level(df, variant_name, target_level):
    # Ищем строку с данным названием варианта
    current_row = df[df["id"] == variant_name]
    if current_row.empty:
        return "HISTONE_NOT_EXIST"

    if current_row["level"].values[0] == target_level:
        return current_row["id"].values[0]

    # Пока не найден родитель с нужным уровнем
    while not current_row.empty:
        # Получаем значение поля parent
        parent_id = current_row["parent"].values[0]

        if pd.isna(parent_id):
            print("here")
            return None  # Нет родителя

        # Ищем родителя
        parent_row = df[df["id"] == parent_id]

        # Проверяем уровень родителя
        if parent_row["level"].values[0] == target_level:
            return parent_row["id"].values[0]

        # Продолжаем искать родителя на следующем уровне
        current_row = parent_row

    return None  # Если не найдено


# Пример использования функции

query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

variant_name = "macroH2A.1"
target_level = "variant_group"
parent_name = find_parent_by_level(histone_df, variant_name, target_level)

print(
    f"Родительский вариант для {variant_name} с уровнем {target_level}: {parent_name}"
)

Родительский вариант для macroH2A.1 с уровнем variant_group: HISTONE_NOT_EXIST


In [ ]:
query = "SELECT id FROM histone WHERE parent IS NULL"
cursor.execute(query)
types = cursor.fetchall()
variants_tree = {}
for t, *_ in types:
    variants_tree[t] = get_tree_dict(cursor, t)

## Сравним полученную таблицу с histones.csv

**Вывод:** Будем считать сгенерированную таблицу из БД итоговой, то есть значения в поле `variant`, которые ранее содержали `__???`, будут пустыми, а также не будет `?` в поле `references`. Возможно стоит добавить итоговую сортировку так, чтобы записи вариантов были в том же порядке, что и в дереве.

In [53]:
df_histonescsv = pd.read_csv(
    "/home/l_singh/_scratch/hdb/project_dir/histonedb/CURATED_SET/histones.csv"
).fillna("")
df_histonescsv["taxonomy_id"] = (
    pd.to_numeric(df_histonescsv["taxonomy_id"], errors="coerce")
    .fillna(np.nan)
    .astype("Int64")
)
df_histonescsv["references"] = [
    r.replace("?", "") for r in df_histonescsv["references"]
]
df_histonescsv["references"] = [
    r.replace("()", "") for r in df_histonescsv["references"]
]
df_histonescsv["doublet"] = [""] * df_histonescsv.shape[0]

In [54]:
# Функция для преобразования строкового представления вещественных чисел в строковые целые
def convert_to_integer_string(value):
    if value != "":  # Если значение не пустое
        return str(
            int(float(value))
        )  # Преобразуем в float, затем в int, и снова в строку
    return value  # Если значение пустое, оставляем его без изменений


# Применяем функцию к колонке ncbi_gene_id
df_histonescsv["ncbi_gene_id"] = df_histonescsv["ncbi_gene_id"].apply(
    convert_to_integer_string
)

In [55]:
df_histonescsv[df_histonescsv["accession"] == "HISTDB_H2A_L_0"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
153,HISTDB_H2A_L_0,H2A,short_H2A,H2A.L.1_(Homo_sapiens),,,115482714,H2AL1Q,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,29549088 35099534,MAGNKHSRSSCKPRRQCLSRSRRAELQFPVSHMERCLREGQYARHL...


In [56]:
df_histonescsv.columns

Index(['accession', 'type', 'variant_group', 'variant', 'doublet', 'gi',
       'ncbi_gene_id', 'hgnc_gene_name', 'taxonomy_id', 'organism', 'phylum',
       'class', 'taxonomy_group', 'info', 'references', 'sequence'],
      dtype='object')

In [57]:
sequence_df.columns

Index(['accession', 'type', 'variant_group', 'variant', 'doublet', 'gi',
       'ncbi_gene_id', 'hgnc_gene_name', 'taxonomy_id', 'organism', 'phylum',
       'class', 'taxonomy_group', 'info', 'references', 'sequence'],
      dtype='object')

In [58]:
# Сначала проверим, совпадает ли набор столбцов
set(df_histonescsv.columns) == set(sequence_df.columns)

True

In [59]:
# Функция для сортировки числовых элементов в колонке 'references', оставляя нечисловые элементы на месте
def sort_references_column(df, column_name="references"):
    def sort_reference_string(reference_str):
        elements = reference_str.split()  # Разделяем строку на элементы
        numeric_elements = sorted(
            [el for el in elements if el.isdigit()], key=int
        )  # Сортируем числовые элементы
        non_numeric_elements = [
            el for el in elements if not el.isdigit()
        ]  # Не числовые элементы остаются на месте
        return " ".join(
            numeric_elements + non_numeric_elements
        )  # Объединяем отсортированные числа и нечисловые

    # Применяем функцию к каждому элементу колонки
    df[column_name] = df[column_name].apply(sort_reference_string)
    return df


# Применяем сортировку к колонке 'references' в обеих таблицах
df_histonescsv = sort_references_column(df_histonescsv)
sequence_df = sort_references_column(sequence_df)

In [60]:
# Сортируем строки по всем столбцам
df1_sorted = df_histonescsv.sort_values(by=list(df_histonescsv.columns)).reset_index(
    drop=True
)
df2_sorted = sequence_df.sort_values(by=list(sequence_df.columns)).reset_index(
    drop=True
)

# Сравниваем отсортированные DataFrame
df1_sorted.equals(df2_sorted)

False

In [61]:
df_histonescsv.dtypes

accession         object
type              object
variant_group     object
variant           object
doublet           object
gi                object
ncbi_gene_id      object
hgnc_gene_name    object
taxonomy_id        Int64
organism          object
phylum            object
class             object
taxonomy_group    object
info              object
references        object
sequence          object
dtype: object

In [62]:
sequence_df.dtypes

accession         object
type              object
variant_group     object
variant           object
doublet           object
gi                object
ncbi_gene_id      object
hgnc_gene_name    object
taxonomy_id        Int64
organism          object
phylum            object
class             object
taxonomy_group    object
info              object
references        object
sequence          object
dtype: object

In [63]:
# перед сравнением попробуем удалить записи из таблиц, где все значения type, variant_group и variant равны пустому значению или
# содержат символы __???
pattern = r"__\?\?\?"  # Шаблон для поиска '__???'
# df_histonescsv_ = df_histonescsv[
#     ~df_histonescsv["type"].str.contains(pattern, regex=True)
#     & ~df_histonescsv["variant_group"].str.contains(pattern, regex=True)
#     & ~df_histonescsv["variant"].str.contains(pattern, regex=True)
# ]

# Удаляем '__???' из указанных колонок
df_histonescsv_ = df_histonescsv.copy()
df_histonescsv_[["type", "variant_group", "variant"]] = df_histonescsv_[
    ["type", "variant_group", "variant"]
].replace(r"__\?\?\?$", "", regex=True)

sequence_df_ = sequence_df[
    (sequence_df["type"] != "")
    & (sequence_df["variant_group"] != "")
    & (sequence_df["variant"] != "")
]

In [64]:
sequence_df_[sequence_df_["variant"] == ""]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence


In [65]:
df_histonescsv_[df_histonescsv_["variant"].str.contains(pattern)]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence


In [66]:
def find_differences(df1, df2):
    # Используем merge с индикатором, чтобы отследить источник каждой строки
    comparison = df1.merge(df2, how="outer", indicator=True)

    # Строки, которые есть только в первой таблице (df1)
    only_in_df1 = comparison[comparison["_merge"] == "left_only"]

    # Строки, которые есть только во второй таблице (df2)
    only_in_df2 = comparison[comparison["_merge"] == "right_only"]

    return only_in_df1, only_in_df2

In [67]:
only_in_df1, only_in_df2 = find_differences(df_histonescsv_, sequence_df_)

In [68]:
# Записи, которые есть только в первой таблице (df_histonescsv):
only_in_df1_ = only_in_df1.drop(columns=["_merge"])
only_in_df1_

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
237,XP_005640164.1,H2B,H2B.1,cH2B_(Mammalia),,545554624,,,9615,Canis lupus familiaris,Chordata,Mammalia,,,,MPELTSKGTTISKKGFKRAVAKTQKKEGKKRRRCRKESYSIYIYKV...
238,NP_783594.1,H2B,H2B.1,cH2B_(Mammalia),,28316750,,,10090,Mus musculus,Chordata,Mammalia,,,,MPEVAVKGATISKKGFKKAVTKTQKKEGRKRKRCRKESYSIYIYKV...
239,NP_072169.1,H2B,H2B.1,cH2B_(Mammalia),,12025524,,,10116,Rattus norvegicus,Chordata,Mammalia,,,,MPEVSAKGTTISKKGFKKAVTKTQKKEGRKRKRCREESYSIYIYKV...
261,NP_835502.1,H2B,cH2B.1_(Mus_musculus),cH2B.1_(Mus_musculus),,,,,10090,Mus musculus,Chordata,Mammalia,,,24506885,MPEPAKSAPAPKKGSKKAVTKAQKKDGKKRKRSRKESYSVYVYKVL...
262,CAA62299.1,H2B,cH2B.1_(Mus_musculus),cH2B.1_(Mus_musculus),,,,,10090,Mus musculus,Chordata,Mammalia,,,8672246,MPEVAVKGATISKKGFKKAVTKTQKKEGRKRKRCRKESYSIYIYKV...
263,Q64524.3,H2B,cH2B.E_(Mus_musculus),cH2B.E_(Mus_musculus),,,,,10090,Mus musculus,Chordata,Mammalia,,,35099534,MPELAKSAPAPKKGSKKAVTKAQKKDGKKRKRSRKESYSIYVYKVL...
405,NP_059141.1,H2B,cH2B(?),cH2B(?)_(Homo_sapiens),,,54145,H2BC12L,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,,MPEPAKSAPAPKKGSKKAVTKAQKKDGRKRKRSRKESYSVYVYKVL...
462,NP_001358848.1,H3,H3.Y,H3.Y.2_(Homo_sapiens),,,340096,H3Y2,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,20819935,MARTKQTARKATAWQAPRKPLATKAARKRASPTGGIKKPHRYKPGT...
463,HISTDB_H3_Y_0,H3,H3.Y,H3.Y,,NOGI,,,9544,Macaca mulatta,Chordata,Mammalia,,,,ARTKQTARKATNWQAPRKPLATKAAAKRAPPRGGIKKPHRYKPGTQ...
464,HISTDB_H3_Y_1,H3,H3.Y,H3.Y,,NOGI,,,9544,Macaca mulatta,Chordata,Mammalia,,,,ARTKQTARKATNWQAPRKPLATKAPGKRLPPRGGIKKPHRYRPGTQ...


In [69]:
# Записи, которые есть только во второй таблице (sequence_df):
only_in_df2_ = only_in_df2.drop(columns=["_merge"])
only_in_df2_

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
572,AAK39657.1,H3,cenH3_(Eukarya),cenH3_(Plants),,13794280,,,55529,Guillardia theta,,Cryptophyceae,,,,MMKKQNLKRFKKSSNSLVDIRKFQKSTDLLIHRLPFARLVKEISLK...
573,CAA62299.1,H2B,cH2B,cH2B.1_(Mus_musculus),,,,,10090,Mus musculus,Chordata,Mammalia,,,8672246,MPEVAVKGATISKKGFKKAVTKTQKKEGRKRKRCRKESYSIYIYKV...
574,HISTDB_H3_Y_0,H3,H3.3-like?,H3.Y_(Primates?),,NOGI,,,9544,Macaca mulatta,Chordata,Mammalia,,,,ARTKQTARKATNWQAPRKPLATKAAAKRAPPRGGIKKPHRYKPGTQ...
575,HISTDB_H3_Y_1,H3,H3.3-like?,H3.Y_(Primates?),,NOGI,,,9544,Macaca mulatta,Chordata,Mammalia,,,,ARTKQTARKATNWQAPRKPLATKAPGKRLPPRGGIKKPHRYRPGTQ...
576,HISTDB_H3_Y_2,H3,H3.3-like?,H3.Y_(Primates?),,NOGI,,,9598,Pan troglodytes,Chordata,Mammalia,,,,ARTKQTARKATAWQAPRKPLATKAAGKRAPPTGGIKKPHRYKPGTL...
577,HISTDB_H3_Y_3,H3,H3.3-like?,H3.Y_(Primates?),,NOGI,,,9598,Pan troglodytes,Chordata,Mammalia,,,,ARTKQTARKATAWQAPRKPLATKAARKRASPTGGIKKPHRYKPGTL...
578,NP_001013721.2,H3,H3.5_(Primates_or_Hominids?),H3.5_(Homo_sapiens),,,440093,H3-5,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,21274551,MARTKQTARKSTGGKAPRKQLATKAARKSTPSTCGVKPHRYRPGTV...
579,NP_001035891.1,H3,cenH3_(Eukarya),cenH3_(Homo_sapiens),,,1058,CENPA,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,23324462,MGPRRRSRKPEAPRRRSPSPTPTPGPSRRGPSLGASSHQHSRRRQG...
580,NP_001295191.1,H1,OO_H1.8,H1.8_(Homo_sapiens),,,132243,H1-8,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MAPATAPRRAGEAKGKGPKKPSEAKEDPPNVGKVKKAAKRPAKVQK...
581,NP_001342187.1,H3,H3.3-like?,H3.Y.1_(Homo_sapiens),,,391769,H3Y1,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,20819935,MARTKQTARKATAWQAPRKPLATKAAGKRAPPTGGIKKPHRYKPGT...


In [70]:
df_histonescsv.shape, sequence_df.shape

((572, 16), (569, 16))

# Close connection

In [15]:
cursor.close()
conn.close()
tunnel.stop()